# Spider Walking

In [ ]:
import numpy as np

import gtdynamics as gtd

In [ ]:
from gtdynamics import Phase, WalkCycle, Trajectory

import gtsam
from gtsam import Pose3, Point3, Rot3
from gtsam.noiseModel import Isotropic, Unit
Z_6x1 = np.zeros((6,), float)

import plotly.express as px

In [ ]:
# Load Stephanie's spider robot.
robot = gtd.CreateRobotFromFile(gtd.SDF_PATH + "/spider_alt.sdf", "spider")

In [ ]:
# Noise models.
sigma_dynamics = 1e-5    # std of dynamics constraints.
sigma_objectives = 1e-6  # std of additional objectives.
sigma_joints = 1.85e-4   # 1.85e-4

dynamics_model_6 = Isotropic.Sigma(6, sigma_dynamics)
dynamics_model_1 = Isotropic.Sigma(1, sigma_dynamics)
dynamics_model_1_2 = Isotropic.Sigma(1, sigma_joints)
objectives_model_6 = Isotropic.Sigma(6, sigma_objectives)
objectives_model_1 = Isotropic.Sigma(1, sigma_objectives)

In [ ]:
# Env parameters.
gravity, mu = np.array([0, 0, -9.8]), 1.0

opt = gtd.OptimizerSetting(sigma_dynamics)
graph_builder = gtd.DynamicsGraph(opt, gravity, None)

In [ ]:
# Create the trajectory, consisting of 3 walk cycles, each consisting of 4
# phases: [stationary, odd, stationary, even].
odd_links=["tarsus_1_L1", "tarsus_3_L3", "tarsus_5_R4", "tarsus_7_R2"]
even_links=["tarsus_2_L2", "tarsus_4_L4", "tarsus_6_R3", "tarsus_8_R1"]
links = odd_links + even_links

contact_in_com = np.array([0, 0.19, 0])
stationary = Phase(1, links, contact_in_com)
odd = Phase(2, odd_links, contact_in_com)
even = Phase(2, even_links, contact_in_com)

walk_cycle = WalkCycle()
walk_cycle.addPhase(stationary)
walk_cycle.addPhase(even)
walk_cycle.addPhase(stationary)
walk_cycle.addPhase(odd)

trajectory = Trajectory(robot, walk_cycle, 3)

In [ ]:
# Create multi-phase trajectory factor graph
collocation = gtd.CollocationScheme.Euler
graph = trajectory.multiPhaseFactorGraph(graph_builder, collocation, mu)

# Build the objective factors.
step = np.array([0,0.4,0])
objectives = trajectory.contactPointObjectives(Isotropic.Sigma(3, 1e-7), step)

In [ ]:
# Get final time step.
K = trajectory.getEndTimeStep(trajectory.numPhases() - 1)

# Add base goal objectives to the factor graph.
base_link = robot.link("body")
for k in range(K+1):
    gtd.add_link_objectives(objectives, base_link.id(), k).pose(Pose3(Rot3(), Point3(0, 0.0, 0.5)), Isotropic.Sigma(6, 5e-5)).twist(Z_6x1, Isotropic.Sigma(6, 5e-5))

In [ ]:
# Add link and joint boundary conditions to FG.
trajectory.addBoundaryConditions(objectives, dynamics_model_6,
                                dynamics_model_6, objectives_model_6,
                                objectives_model_1, objectives_model_1)

# Constrain all Phase keys to have duration of 1 /240.
desired_dt = 1. / 240
trajectory.addIntegrationTimeFactors(objectives, desired_dt, 1e-30)

# Add min torque objectives.
trajectory.addMinimumTorqueFactors(objectives, Unit.Create(1))

In [ ]:
# Add prior on hip joint angles (spider specific)
prior_model = Isotropic.Sigma(1, 1.85e-4)
for joint in robot.joints():
    if "hip2" in joint.name():
        for k in range(K+1):
            gtd.add_joint_objectives(objectives, joint.id(), k).angle(2.5, prior_model)

In [ ]:
  # Add objectives to factor graph.
  graph.push_back(objectives)

In [ ]:
# Initialize solution.
gaussian_noise = 1e-5
init_vals = trajectory.multiPhaseInitialValues(gaussian_noise, desired_dt)

In [ ]:
# Optimize!
params = gtsam.LevenbergMarquardtParams()
params.setVerbosityLM("SUMMARY")
params.setlambdaInitial(1e10)
params.setlambdaLowerBound(1e-7)
params.setlambdaUpperBound(1e10)
params.setAbsoluteErrorTol(1.0)
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, init_vals, params)
results = optimizer.optimize()

In [ ]:
graph.error(results)

In [ ]:
trajectory.writeToFile("spider.csv", results);
